https://thebook.io/080324/part05/ch16/03/

3~5. 컨볼루션 신경망 (CNN Convolutional Neural Network)

cf. 이미지 분야에서

- 영역처리(Convolution)    
ai-kernal, mask images-filter...   
커널을 통해 값 계산한 값을 계산 범위 가운데에 넣음 -> 결과 값의 사이즈는 줄어든다   
(상하좌우 픽셀이 빠지기때문 잘리는 픽셀의 크기는 커널크기에 따라서 다름) 커널 규모는 보통 홀수로   



마스크의 값에 따라 여러효과 가능   
1. Blurring   
급격하게 변하는 부분을 점진적으로 바꿔줌 (부드럽)   
2. Sharpening   
점진적으로 변하는 부분을 급격하게 바꿔줌 (또렷)    
3. Edge   
edge(테두리)검출 가능   
Canny edge detection   
4. Filtering   
...

In [1]:
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = '1'


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype(float) / 255 
# X_train.shape[0], 28, 28, (1)=>채널수 컬러그래픽의 경우 3
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype(float) / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
#Conv2D(32) 1장 기준 32장의 2d 이미지 출력
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
#Conv2D(64) 1장 기준 64장의 2d 이미지 출력
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                      

1. model.add(layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))   
1x(3x3)x32 + 32 = **320**   


2. model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))   
32x(3x3)x64 + 64 = **18,496**   


3. model.add(layers.MaxPooling2D(pool_size=(2, 2)))   
output shape 12x12x1   


4. model.add(layers.Dropout(0.25))   
파라미터 영향x   


5. model.add(layers.Flatten())   
output (12x12x1) x 64 = 9,216   


6. model.add(layers.Dense(128, activation='relu'))   
9216 x 128 = 1,179,648 + 128 = **1,179,776**   


7. model.add(layers.Dropout(0.25))   
파라미터 영향x   


8. model.add(layers.Dense(10, activation='softmax'))   
128x10 + 10 = **1,290**


[시트 정리](Chap16_cnn_param.xlsx)

![16.png](16.png)

##### model.add(layers.Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 1), activation='relu'))
1 | 첫 번째 인자: 커널을 몇 개 적용할지 정합니다. 여기서는 32개의 커널을 적용했습니다.

2 | kernel_size: 커널의 크기를 정합니다. kernel_size=(행, 열) 형식으로 정하며, 여기서는 3×3 크기의 커널을 사용하게끔 정했습니다.

3 | input_shape: Dense 층과 마찬가지로 맨 처음 층에는 입력되는 값을 알려 주어야 합니다. input_shape=(행, 열, 색상 또는 흑백) 형식으로 정합니다. 만약 입력 이미지가 색상이면 3, 흑백이면 1을 지정합니다. 여기서는 28×28 크기의 흑백 이미지를 사용하도록 정했습니다.

4 | activation: 사용할 활성화 함수를 정의합니다.

##### model.add(layers.MaxPooling2D(pool_size=(2, 2)))
정해진 구역 안에서 최댓값을 뽑아내는 맥스 풀링(max pooling) shape는 줄이지만 특징은 가져감   
(2,2)는 가로 2, 세로 2 크기의 풀링 창을 통해 맥스 풀링을 진행하라는 의미

##### model.add(layers.Dropout(0.25))
드롭아웃은 은닉층에 배치된 노드 중 일부를 임의로 꺼 주는 것=>과대적합 방지   
25%의 노드를 끄는 코드

##### model.add(layers.Flatten())
2차원 배열을 1차원으로 Dense 이용하기 위해서